In [16]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

In [17]:
#établissement de la connexion à la base de données et création du curseur
conn = sqlite3.connect("haikintana.db")
c= conn.cursor()

In [18]:
#création des tables
c.execute("create table if not exists membres (ID text Primary Key, cotisation text, nom text, prénoms text, email text, bénévolat text, statut text, assiduité text)")
c.execute("create table if not exists activité(intitulé text Primary Key, date text, lieu text)")
c.execute("create table if not exists pôle(nom_pôle text Primary Key, nombre_activités integer)")

In [29]:
#importation des données à partir du fichier excel et transfert vers la base
data=pd.read_excel('C:/Users/Harivola RABEKOTO/Liste HAIKINTANA 2023.xlsx')
data=data.drop(['Pôles','Commentaire'], axis=1) #on ne prend pas en compte les deux dernières colonnes du fichier excel
data=data.rename(columns={'Numéro':'ID'}) #on change le label 'numéro' en 'ID' pour faire correspondre cette colonne à celle de la table
print(data)
data.to_sql('membres', conn, if_exists='replace',index=False)

         ID                  Cotisation                        Nom  \
0   HK-0000                         NaN                 RAZANAKOTO   
1   HK-0001                         NaN              RAJAONARIVELO   
2   HK-0002                         NaN                Ralambomamy   
3   HK-0003                         NaN             Rafieferantsoa   
4   HK-0004                         NaN                  Rabarison   
5   HK-0005                        payé             Andriamandroso   
6   HK-0006                         NaN                  Rakotoson   
7   HK-0007                   durant AG                    RABEONY   
8   HK-0008                   durant AG          Randriamanantena    
9   HK-0009                   durant AG           RANDRIAMANANTENA   
10  HK-0010               mvola(09mars)                 Rambeloson   
11  HK-0011                   durant AG            IALISOLONDRAIBE   
12  HK-0012                   durant AG         randrianandrasana    
13  HK-0013         

In [20]:
#remplissage de la table pôle
liste_pôles=[
    ('éducation',0),
    ('outreach',0),
    ('observation et instrumentation',0),
    ('développement et partenariat',0)
]
c.executemany("insert into pôle values(?,?)",liste_pôles)

IntegrityError: UNIQUE constraint failed: pôle.nom_pôle

In [34]:
#remplacement des valeurs nulles en un texte "aucun"
for i in c.execute('select* from membres'):
    for j in range (7):
        if isnull(i[j]):
            c.execute('update membres set i[j]= 'Aucun' ')

SyntaxError: invalid syntax (<ipython-input-34-9463c84802ed>, line 5)

In [31]:
#affichage d'informations
def info_membre(id_recherche):
    c.execute("select * from membres where (ID)= (?)",(id_recherche,))
    informations_membre=c.fetchall()
    #using for loop to move out of the list 
    for i in informations_membre:
        print('Nom: '+ i[2] + '\n'+
              'Prénoms: '+ i[3] + '\n'+
             'Contact: '+ i[4] + '\n'+
             'Cotisation: '+ i[1] + '\n'+
             'Bénévolat: '+ i[5] + '\n'+
             'Statut: '+ i[6] + '\n' +
             'Nombre d\'activités assistées: '+ str(i[7]) + '\n')
info_membre('HK-0000')

TypeError: can only concatenate str (not "NoneType") to str

In [32]:
#vérification base de données
for row in c.execute("select * from membres"):
    print(row)
for row in c.execute("select * from pôle"):
    print(row)

('HK-0000', None, 'RAZANAKOTO', 'Mializo', None, None)
('HK-0001', None, 'RAJAONARIVELO', 'Andoniaina', None, None)
('HK-0002', None, 'Ralambomamy', 'Tyffani', None, None)
('HK-0003', None, 'Rafieferantsoa', 'Deralaza ', None, None)
('HK-0004', None, 'Rabarison', 'Toky ', None, None)
('HK-0005', 'payé', 'Andriamandroso', 'Mianoka', 'mianoka@homemade.mg', None)
('HK-0006', None, 'Rakotoson', 'Ny Tsiky', 'rakotoson.ny.tsiky.m@gmail.com', None)
('HK-0007', 'durant AG', 'RABEONY', 'Miora Heritiana', 'miorahrabeony@gmail.com', 'Non')
('HK-0008', 'durant AG', 'Randriamanantena ', 'Herman ', 'herman.andriam@gmail.com', 'Oui')
('HK-0009', 'durant AG', 'RANDRIAMANANTENA', 'Volasoa Elia', 'eliavrandria@gmail.com', 'Oui')
('HK-0010', 'mvola(09mars)', 'Rambeloson', 'Koloina Razafinjato', 'rambelosonkoloina@gmail.com', 'Oui')
('HK-0011', 'durant AG', 'IALISOLONDRAIBE', 'Jean Miguel Haydon', 'miguelhaydon@gmail.com', 'Oui')
('HK-0012', 'durant AG', 'randrianandrasana ', 'rojotiana daniel', 'rojotian

In [11]:
#fermeture de la connexion
conn.close()